In [101]:
import torch # For tensor operations
from torchvision.datasets import MNIST
import torchvision.transforms as transforms # Transformations we can perform on our dataset for augmentation
from torch import optim # For optimizers like SGD, Adam, etc.
from torch import nn # To inherit our neural network
from torch.utils.data import DataLoader # For management of the dataset (batches)

In [102]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()

        self.w1 = nn.Linear(input_size,hidden_size)
        self.w2 = nn.Linear(hidden_size,output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.w1(x)
        x = self.sigmoid(x)
        x = self.w2(x)
        return x

In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 784
hidden_size = 100
out_size = 10
learning_rate = 0.01
batch_size = 100
epochs = 50

In [104]:
train_dataset = MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)

In [105]:
model = NN(input_size, hidden_size, out_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [106]:
for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        data = data.reshape(data.shape[0], -1)

        scores = model(data)
        loss = criterion(scores,targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}")

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [107]:
predictions_list = []

In [108]:
import pandas as pd
def check_accuracy(loader, model, test=False):
    global predictions_list

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            if test:
                predictions_list += predictions.tolist()

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    
    model.train()
    return num_correct / num_samples

In [109]:
print(f"Accuracy at training: {check_accuracy(train_loader,model)*100:.4}%")
print(f"Accuracy at testing: {check_accuracy(test_loader,model, test=True)*100:.4}%")

Accuracy at training: 99.68%
Accuracy at testing: 97.12%


Output:

In [110]:
#output in csv
import pandas as pd

data = {
    "ID": [],
    "target": [],
}

for i, label in enumerate(predictions_list):
    data["ID"].append(i)
    data["target"].append(label)

df = pd.DataFrame(data)
df.to_csv("submission.csv", index=False)